In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import sequence

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

17464789/17464789 [==============================] - 0s 0us/step


In [4]:
word_index = tf.keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


In [5]:
word_index = {k:(v+3) for k, v in word_index.items()}

word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [6]:
# 這邊可以調
review_length = 500

x_train = sequence.pad_sequences(x_train, maxlen = review_length)
x_test = sequence.pad_sequences(x_test, maxlen = review_length)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(25000, 500)
(25000,)
(25000, 500)
(25000,)


In [7]:
from tensorflow.keras import layers, models


In [8]:
dnn = models.Sequential(
    [
        layers.Embedding(
            input_dim = vocab_size,
            output_dim = 32,
            input_length = review_length,
        ),

        layers.Dense(
            units = 32,
        ),

        layers.Dense(
            units = 1,
        ),
    ],
    name = "DNN"
)

dnn.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = ["accuracy"],
)

dnn.summary()

Model: "DNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 dense (Dense)               (None, 500, 32)           1056      
                                                                 
 dense_1 (Dense)             (None, 500, 1)            33        
                                                                 
Total params: 321089 (1.22 MB)
Trainable params: 321089 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
history = dnn.fit(
    x_train,
    y_train,
    batch_size = 250,
    epochs = 10,
    validation_split = 0.2,
)

Epoch 1/10
80/80 [==============================] - 67s 814ms/step - loss: 1.1589 - accuracy: 0.4985 - val_loss: 0.7633 - val_accuracy: 0.5061
Epoch 2/10
80/80 [==============================] - 68s 857ms/step - loss: 0.7273 - accuracy: 0.4988 - val_loss: 0.7025 - val_accuracy: 0.5043
Epoch 3/10
80/80 [==============================] - 67s 832ms/step - loss: 0.6990 - accuracy: 0.4997 - val_loss: 0.6965 - val_accuracy: 0.4960
Epoch 4/10
80/80 [==============================] - 65s 818ms/step - loss: 0.6959 - accuracy: 0.5005 - val_loss: 0.6952 - val_accuracy: 0.4955
Epoch 5/10
80/80 [==============================] - 66s 823ms/step - loss: 0.6949 - accuracy: 0.5007 - val_loss: 0.6945 - val_accuracy: 0.4965
Epoch 6/10
80/80 [==============================] - 78s 983ms/step - loss: 0.6944 - accuracy: 0.5028 - val_loss: 0.6943 - val_accuracy: 0.4944
Epoch 7/10
80/80 [==============================] - 60s 749ms/step - loss: 0.6941 - accuracy: 0.5008 - val_loss: 0.6938 - val_accuracy: 0.4967

In [10]:
import json

history_dict = history.history

In [11]:
json.dump(history_dict, open("/content/drive/MyDrive/Colab Notebooks/imdb/history/dnn.json", 'w'))

In [12]:
dnn.save("/content/drive/MyDrive/Colab Notebooks/imdb/model/dnn.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
